In [88]:
from langchain.chat_models import init_chat_model
llm= init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [89]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [90]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7caa98b82ac0>)

In [91]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [92]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [93]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [94]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [95]:
exponentiate.func(**llm_output_dict)

25

## Creating an Agent

We're going to construct a simple tool calling agent. We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's _"notes"_ as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [96]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [97]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

Now we will initialize our agent. For that we need:

* `llm`: as already defined
* `tools`: to be defined (just a list of our previously defined tools)
* `prompt`: as already defined
* `memory`: as already defined

In [98]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

Our `agent` by itself is like one-step of our agent execution loop. So, if we call the `agent.invoke` method it will get the LLM to generate a single response and go no further, so no tools will be executed, and no next iterations will be performed.

We can see this by asking a query that should trigger a tool call:

In [99]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"x": 10.7, "y": 7.68}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--25c79196-b612-4afb-9e78-fc44e5f1ebe8-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': '9eb39d5b-2aa5-4e51-9507-0ab565af2d53', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 5, 'total_tokens': 96, 'input_token_details': {'cache_read': 0}})], tool_call_id='9eb39d5b-2aa5-4e51-9507-0ab565af2d53')]

In [100]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [101]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.1759999999999910.  7 multiplied by 7.68 is 82.176.


> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.  7 multiplied by 7.68 is 82.176.\n', additional_kwargs={}, response_metadata={})],
 'output': '10.  7 multiplied by 7.68 is 82.176.\n'}

Let's test our agent with some memory and tool use. First, we tell it our name, then we will perform a few tool calls, then see if the agent can still recall our name.

First, give the agent our name:

In [102]:
agent_executor.invoke({
    "input": "My name is James",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Nice to meet you, James!


> Finished chain.


{'input': 'My name is James',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.  7 multiplied by 7.68 is 82.176.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James!\n', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, James!\n'}

In [103]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 9.0, 'y': 10.0}`


19.0
Invoking: `multiply` with `{'x': 4.0, 'y': 2.0}`


8.0
Invoking: `exponentiate` with `{'x': 8.0, 'y': 3.0}`


512.0
Invoking: `subtract` with `{'x': 512.0, 'y': 19.0}`


-493.0Nine plus 10, minus 4 * 2, to the power of 3 is -493.


> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.  7 multiplied by 7.68 is 82.176.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James!\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nine plus 10, minus 4 * 2, to the power of 3 is -493.\n', additional_kwargs={}, response_metadata={})],
 'output': 'Nine plus 10, minus 4 * 2, to the power of 3 is -493.\n'}

In [104]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory})



> Entering new AgentExecutor chain...
Your name is James.


> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.  7 multiplied by 7.68 is 82.176.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James!\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nine plus 10, minus 4 * 2, to the power of 3 is -493.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is James.\n', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is James.\n'}

In [105]:
import os
from getpass import getpass
# Force prompt for a new key
os.environ["SERPAPI_API_KEY"] = getpass("Enter your new SerpAPI API key: ")

In [106]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

These custom tools can look into your IP address, find out where you are currently, then we will also use a secondary function to get the current date and time, then we will use this information to feed into the SerpAPI to find us the weather pattern in your area and at the time of the function calling.

In [107]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

We can create our prompt, this time we'll skip the `chat_history` part as we don't need it. However, you can add it if preferred.

In [108]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [109]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

In [110]:
agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

In [113]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather in paris now? Please give me degrees in Celsius"
    )
})
from IPython.display import display, Markdown

display(Markdown(out["output"]))



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2025-06-18 01:09:05
Invoking: `Search` with `weather in Paris`


{'type': 'weather_result', 'temperature': '66', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '65%', 'wind': '4 mph', 'location': 'Paris, France', 'date': 'Wednesday 2:00 AM', 'weather': 'Partly cloudy'}Right now it is 2025-06-18 01:09:05. The weather in Paris is partly cloudy with a temperature of 66 degrees Fahrenheit, 0% chance of precipitation, 65% humidity, and 4 mph wind.

66 degrees Fahrenheit is about 19 degrees Celsius.


> Finished chain.


Right now it is 2025-06-18 01:09:05. The weather in Paris is partly cloudy with a temperature of 66 degrees Fahrenheit, 0% chance of precipitation, 65% humidity, and 4 mph wind.

66 degrees Fahrenheit is about 19 degrees Celsius.
